In [1]:
print("all okay")

all okay


In [2]:
from langchain_groq import ChatGroq

In [3]:
from dotenv import load_dotenv
load_dotenv()

True

In [4]:
model = ChatGroq(model='qwen/qwen3-32b')

In [5]:
query = "What is the capital of France?"
response = model.invoke(query).content

print(f"Response: {response}")


Response: <think>
Okay, so the user is asking, "What is the capital of France?" Hmm, I need to make sure I get this right. France is a country in Europe, right? I remember from school that the capital is Paris. But wait, maybe I should double-check. Sometimes countries have cities with similar names, but France is pretty straightforward.

Let me think. Paris is known for the Eiffel Tower and the Louvre, and it's a major city. Yeah, I'm pretty sure it's Paris. But maybe there's a trick here? Like, some people might confuse it with Lyon or Marseille, but those are other cities in France. No, the capital is definitely Paris. 

Wait, has there ever been a time when the capital wasn't Paris? I don't think so. Even during different historical periods, Paris has remained the capital. Unless there was some temporary move, but that's not common knowledge. So, the answer should be Paris. 

I should also consider if the user is expecting any other information, like the population or landmarks, bu

In [6]:
from langchain_google_genai import GoogleGenerativeAIEmbeddings

In [8]:
embedding_model = GoogleGenerativeAIEmbeddings(model='models/embedding-001',)

In [10]:
emd = embedding_model.embed_query("What is the capital of France?")
len(emd)

768